In [1]:
!pip install tensorflow

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

DATASET_PATH_train = ""  # Update with the actual dataset path
DATASET_PATH_valid= ""  # Update with the actual dataset path
IMG_SIZE = (224, 224)           

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    DATASET_PATH_train,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='binary'
)

validation_generator = train_datagen.flow_from_directory(
    DATASET_PATH_valid,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='binary'
)

class CNN(tf.keras.Model):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1_filters = tf.Variable(tf.random.truncated_normal([3, 3, 3, 32], stddev=0.1), trainable=True,regularizer=l2(0.01))
        self.conv2_filters = tf.Variable(tf.random.truncated_normal([3, 3, 32, 64], stddev=0.1), trainable=True,regularizer=l2(0.01))
        self.conv3_filters = tf.Variable(tf.random.truncated_normal([3, 3, 64, 128], stddev=0.1), trainable=True,regularizer=l2(0.01))
        
        self.fc1 = Dense(128, activation='relu', kernel_regularizer=l2(0.01))
        self.dropout = Dropout(0.8)
        self.output_layer = Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01))
    
    def call(self, x, training=False):
        x = tf.nn.conv2d(x, self.conv1_filters, strides=[1, 1, 1, 1], padding='SAME')
        x = tf.nn.relu(x)
        x = tf.nn.max_pool2d(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        x = tf.nn.conv2d(x, self.conv2_filters, strides=[1, 1, 1, 1], padding='SAME')
        x = tf.nn.relu(x)
        x = tf.nn.max_pool2d(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        x = tf.nn.conv2d(x, self.conv3_filters, strides=[1, 1, 1, 1], padding='SAME')
        x = tf.nn.relu(x)
        x = tf.nn.max_pool2d(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        x = Flatten()(x)
        x = self.fc1(x)
        if training:
            x = self.dropout(x, training=training)
        x = self.output_layer(x)
        
        return x

model = CNN()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator)
model.save("")
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy:.2f}")

import matplotlib.pyplot as plt
import numpy as np

# Plot training and validation accuracy
def plot_accuracy_and_loss(history):
    epochs = range(1, len(history.history['accuracy']) + 1)
    
    # Plot accuracy
    plt.figure(figsize=(14, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history.history['accuracy'], 'bo-', label='Training accuracy')
    plt.plot(epochs, history.history['val_accuracy'], 'r^-', label='Validation accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history.history['loss'], 'bo-', label='Training loss')
    plt.plot(epochs, history.history['val_loss'], 'r^-', label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.show()

plot_accuracy_and_loss(history)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 766.5 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 2.0 MB/s eta 0:00:0000:0100:01
Using cached protobuf-5.29.3-cp38-abi3-macosx_10_9_universal2.whl (417 kB)
Using cached tensorboard-2.18.0-py3-none-any.whl (5.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.3
    Uninstalling numpy-2.2.3:
      Successfully uninstalled numpy-2.2.3


FileNotFoundError: [Errno 2] No such file or directory: ''